# preprocessing

In [1]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
import re
from nltk.corpus import stopwords
stopWords = set(stopwords.words('indonesian'))
import pandas as pd
import numpy as np

In [2]:
def casefold(str):
    return str.casefold()

In [3]:
def tokenisasi(str):
    return tknzr.tokenize(str)

In [4]:
def stopword_removal(token):
    result = []
    for i in range(len(token)):
        if token[i] not in stopWords:
            result.append(token[i])
    return result

In [5]:
def url_symbol_removal(token):
    new_token = []
    for t in token:
        if t[:1] != "@" and t[:1] != "#":
            url_removed = re.sub(r"http\S+", "", t)
            symbol_removed = re.sub(r"[^\w]", "", url_removed)
            if symbol_removed != '':
                new_token.append(symbol_removed)
    return new_token

In [6]:
def preprocessing(string):
    string = string[2:-1]
    a = casefold(string)
    a = tokenisasi(a)
    a = stopword_removal(a)
    a = url_symbol_removal(a)
    a = ' '.join(a)
    return a

In [7]:
data = pd.read_csv("training_labeled.csv")

In [8]:
tweets = data["Tweet"]
label = data["Label"]

In [9]:
result = []
for i in range(len(tweets)):
    pre = preprocessing(tweets[i]) if preprocessing(tweets[i]) != "" else "bagus" if label[i] == 1 else "payah"
    result.append({'Text' : pre,'Label' : label[i]})
df = pd.DataFrame(result)
df.to_csv('preprocessing result.csv', index=False, header='column_names')

# NBC

In [10]:
def likelihood(texts):
    token=[]
    positive={}
    negative={}
    for i in range(len(texts)):
        t = tokenisasi(texts[i])
        for something in t:
            if something not in token:
                token.append(something)
                if labels[i] == 1:
                    positive[something] = 1
                    negative[something] = 0
                else:
                    positive[something] = 0
                    negative[something] = 1
            else:
                if label[i] == 1:
                    positive[something] += 1
                else:
                    negative[something] += 1
    kata = []
    positif = []
    negatif = []
    for key in positive:
        kata.append(key)
        positif.append(positive[key])
        negatif.append(negative[key])
    res = pd.DataFrame({
        "kata" : kata,
        "positif" : positif,
        "negatif" : negatif
    })
    return res,token,positif,negatif,positive,negative    

In [11]:
def prob(w,c,token,positif,negatif,positive,negative):
    if c == "positif":
        if w not in token:
            return (0+1)/(sum(positif)+len(token))
        else:
            return (positive[w]+1)/(sum(positif)+len(token))
    elif c == "negatif":
        if w not in token:
            return (0+1)/(sum(negatif)+len(token))
        else:
            return (negative[w]+1)/(sum(negatif)+len(token))

In [12]:
def P(text,token,positif,negatif):
    words = tokenisasi(text)
    positive_probability = np.prod([prob(word,"positif",token,positif,negatif,positive,negative) for word in words]+[prior_positif])
    negative_probability = np.prod([prob(word,"negatif",token,positif,negatif,positive,negative) for word in words]+[prior_negatif])
    return "positif" if positive_probability > negative_probability else "negatif"

In [13]:
def Classification(length_training,likelihood,testing):
    TP,FP,TN,FN =0,0,0,0
    precision,recall,F1,accuracy=0,0,0,0
    unique, counts = np.unique(testing['label'],return_counts=True)
    prior_positif = dict(zip(unique,counts))['positif']/100
    prior_negatif = dict(zip(unique,counts))['negatif']/100
    result = {"kalimat":[],"label":[]}
    for i in range(len(testing)):
        prediction = P(texts[i],token,positif,negatif)
        if prediction=="positif":
            if testing['label'][i]=="positif":
                TP = TP+1
            else:
                FP = FP+1
        else:
            if testing['label'][i]=="negatif":
                TN = TN+1
            else:
                FN=FN+1
    precision=TP/(TP+FP)
    recall=TP/(TP+FN)
    F1=(2*precision*recall)/(precision+recall)
    accuracy=(TP+TN)/(TP+FP+TN+FN)
    return TP,FP,TN,FN,precision,recall,F1,accuracy

In [14]:
data_preprocessing = pd.read_csv('preprocessing result.csv')
texts = data_preprocessing["Text"]
labels = data_preprocessing["Label"]

In [15]:
res,token,positif,negatif,positive,negative   = likelihood(texts)

In [16]:
res.to_csv("likelihood.csv",index=False)

In [17]:
unique, counts = np.unique(labels,return_counts=True)
prior_positif = dict(zip(unique,counts))[1]/100
prior_negatif = dict(zip(unique,counts))[0]/100

In [18]:
result = {"kalimat":[],"label":[]}
for i in range(len(texts)):
    result['kalimat'].append(texts[i])                       
    result['label'].append(P(texts[i],token,positif,negatif))

In [19]:
res = pd.DataFrame(result)

In [20]:
res.to_csv("prediction.csv", index=False, header='column_names')

In [21]:
pd.read_csv("prediction.csv")

kalimat    label
0                                  orang disampingnya  positif
1                                  demokratnya mah gk  negatif
2   wowo anak buah kayak dancok untung kau yg dian...  negatif
3   lucu gue liat d yg dukung orng ndtang d ilc bi...  negatif
4   muak dgn tingkah 2 pengumpat kubu puasa ramadh...  positif
5   insya allah ayahanda tulus jujur jujur membela...  positif
6   rakyat bodoh dungu sj ndpt dipengaruhi elit po...  positif
7                 klo drmokrat tdk keprntingan lg kah  positif
8                       pa gausah turun biar emak aja  positif
9   02 gk suka tdk menganggp lbih yg kalah jk kalh...  positif
10                                            jebakan  negatif
11  artikan power kekuatan fisik kekuatan moral po...  positif
12  njangan offline side n ndarah merah pemakan da...  positif
13  senin bang mengadakan buka puasa rumah kertane...  positif
14                         jd gunanya bawaslu mk dkkp  positif
15                                       bentuklah om  positif
16  resiko pembangkangan rakyat semesta mayoritas ...  positif
17  pokoknya prabowo menang nharus jd presiden n n...  positif
18                        pokeke lawan 2024 yg berani  positif
19                           gorengan xf0 x9f x98 x81  positif
20  pake xe2 x80 x9crakyat xe2 x80 x9d tolong dita...  positif
21                                              bagus  positif
22  nmassa andaanda nkedaulatan konstitusi terarah...  negatif
23  alhamdulillah puisi nw munajat akbar 212 dikab...  positif
24  murka allah diaceh bencana alam sunami 26 des ...  negatif
25  menang 2014 curang klo 2019 tingal diam hadapi...  negatif
26    sok bener dik nmau manganulir pemilu 2019 nnaif  negatif
27  gila keracunan kebanyakan kekuasaan kebodohan ...  negatif
28                  ahy ngaku kalah krn urutan ketiga  negatif
29      emang ahmad dani yg ditahan xf0 x9f x98 x82 n  negatif
..                                                ...      ...
70  penjarakan nmanusia pengangguran politik nhany...  negatif
71  diceneralisasi pilpres curang kpu curang bohon...  negatif
72                   win win solution maksud loh grey  positif
73        kerennnn om lanjutkan om semangat tersenyum  positif
74  baca menduga pernyataan hasto fitnah mengarah ...  positif
75  jujur xf0 x9f xa4 x94 xf0 x9f x98 x81 xf0 x9f ...  negatif
76  slah calon uang rakyat menggunakn lembaga nega...  positif
77                                       cc nbang bro  positif
78                                           cc in ah  positif
79  ga pake ahy bang pideo nya promosi ya xf0 x9f ...  negatif
80  tujuan kubu 02 ngusulin pembentukan tpf utk me...  positif
81  mengaku kalah ga pakai alasan ngeles ngeles pe...  negatif
82                               tk sutradara tk rela  positif
83  bikin tpf akal akalan yg kalah aja pa jkw gabu...  negatif
84  persengkolan parpol pendukung prabowo mengalih...  positif
85                                              bagus  positif
86         pengkhianat nyinyirlah malu mengangkang 02  negatif
87  indonesia kandung njika biarkan kandung perkos...  positif
88                               pemarah mudah stroke  negatif
89  salut abang mari berjuang bang anak medan kena...  positif
90  alhamdulillah nbukannya ntu tanah leluhur bpk ...  negatif
91  ukri bocorkan 01 dipaksakan menang n n n n n n...  negatif
92  sampeyan kalah suara coba pakai jurrus cawet a...  negatif
93                              kali setuju dgn tweet  positif
94                                   suaranya g kawan  negatif
95                                             mantap  positif
96  dibawah dikecamatan dikawal polisi saksidikeca...  positif
97  netral smnjak liat qc n xf0 x9f x98 x82 xf0 x9...  positif
98                                      alhamdulillah  positif
99                           masjidnya prabowo yg yah  positif

[100 rows x 2 columns]

# Validation

cross validation

In [22]:
def crossValidations(kfold,data):
    result = {'Fold':[],'TP':[],'FP':[],'TN':[],'FN':[],'precision':[],'recall':[],'F1':[],'accuracy':[]}
    fold = round(len(data) / kfold)
    for i in range(kfold):
        start = i * fold
        end = (i + 1) * fold
        training_set = {'kalimat': [],'label':[]}
        testing_set = {'kalimat': [],'label':[]}

        if end > len(data):
            end = len(data)
        for j in range(len(data)):
            if start<= j <= end:
                testing_set['kalimat'].append(data['kalimat'][j])
                testing_set['label'].append(data['label'][j])
            else:
                training_set['kalimat'].append(data['kalimat'][j])
                training_set['label'].append(data['label'][j])
        training = pd.DataFrame(training_set)
        testing = pd.DataFrame(testing_set)
        res,token,positif,negatif,positive,negative = likelihood(training['kalimat'])
        frequencyWord = pd.DataFrame(res)
        TP, FP, TN, FN,precision,recall,F1,accuracy = Classification(len(training),frequencyWord,testing)
        result['Fold'].append(i+1)
        result['TP'].append(TP)
        result['FP'].append(FP)
        result['TN'].append(TN)
        result['FN'].append(FN)
        
        result['precision'].append(precision)
        result['recall'].append(recall)
        result['F1'].append(F1)
        result['accuracy'].append(accuracy)
#         # training.to_csv('../Data/cross_validation/training_fold_'+str(i)+'.csv', index=False)
#         # testing.to_csv('../Data/cross_validation/testing_fold_'+str(i)+'.csv', index=False)
#         # likelihood.to_csv('../Data/cross_validation/likelihood_fold_'+str(i)+'.csv', index=False)
    return result

In [23]:
data_testing = pd.read_csv('prediction.csv')
fold =10

In [24]:
result = crossValidations(10,data_testing)

In [25]:
res = pd.DataFrame(result)

In [26]:
res.to_csv("hasil_cross.csv", index=False, header='column_names')

In [27]:
pd.read_csv("hasil_cross.csv")

Fold  TP  FP  TN  FN  precision    recall        F1  accuracy
0     1   7   0   4   0   1.000000  1.000000  1.000000  1.000000
1     2   6   1   0   4   0.857143  0.600000  0.705882  0.545455
2     3   1   6   1   3   0.142857  0.250000  0.181818  0.181818
3     4   5   2   1   3   0.714286  0.625000  0.666667  0.545455
4     5   4   3   3   1   0.571429  0.800000  0.666667  0.636364
5     6   5   2   2   2   0.714286  0.714286  0.714286  0.636364
6     7   4   3   4   0   0.571429  1.000000  0.727273  0.727273
7     8   4   3   1   3   0.571429  0.571429  0.571429  0.454545
8     9   5   2   3   1   0.714286  0.833333  0.769231  0.727273
9    10   5   2   2   1   0.714286  0.833333  0.769231  0.700000

In [28]:
f1,accuracy=0,0
for i in range(len(result)):
    f1 = f1+result['F1'][i]
    accuracy=accuracy+ result['accuracy'][i]
print("rata-rata F1 ",f1/len(result))
print("rata-rata accuracy ",accuracy/len(result))

rata-rata F1  0.6670279611456081
rata-rata accuracy  0.606060606060606
